# Analyze Twenty One Pilots Lyrics with Python
### Inspired by CodeCademy's "Analyze Taylor Swift Lyrics with Python".

Double check that you have all required packages/libraries installed and accessible before running the import cell. \
Install Seaborn: https://seaborn.pydata.org/installing.html \
Install NLTK (Natural Language ToolKit): https://www.nltk.org/install.html

In [1]:
%matplotlib inline

import pandas as pd
import string
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

### 1. Build and load the dataset
After running the first cell to load necessary libraries, next we'll build and load the dataset ("dataframe") to be analyzed.

Our handiest functions to get insight into our data: \
(See more info on https://pandas.pydata.org/docs/reference/index.html)
- Use pd.read_csv to load the dataframe from a file and save it as a variable dataframe in our jupyter notebook. Use pd.to_csv to write to a file.
>
>Keep in mind as you code that whenever you assign a dataframe to a variable defined in jupyter, any modifications you make to that dataframe stay local to your current jupyter session and do not directly edit the csv file. If you want your edits to the dataframe to be permanent within a file, you have to use pd.to_csv to write to a selected file.
>
>**Be careful when overwriting content!** Check what is in the file before writing to it. You don't want to lose content!
- Use .head() to inspect the first few rows of a dataframe.
- Use .info() to determine how many rows there are, check for missing values, and check the variable types.

How do we get from copy and pasted lyrics only, to a broad dataset with custom information for each line of song lyric?

Below is the process:
1. Copy and paste the lyrics from your chosen song into a new csv file. I've called this "one_song_lyrics_only.csv".
2. Skim the lyrics in the csv file and look for any lines with commas. Commas are the default delimiter, so they will be interpreted not as part of the lyric but as an indicator of a new cell of content. If you use an IDE like Visual Studio Code, extensions like Rainbow CSV can help these jump out immediately. It's your choice how to manage these lines so the dataframe can be properly read and compiled. Here are some options:
   > - My usual preference: simply count every comma in a song's lyrics as a new lyric line, and manually delete it and add a new line. (Just backspace and return after each comma when you see one.) It takes a bit extra time and effort, but I prefer to interpret many song lyrics like this manually.
   > - Wrap the lines with commas in double-quotes. Note that this will force your whole line as a string object, but this should be the default for the rest of your lines anyway. I use this on occasion.
   > - If you want to avoid the issue altogether upfront and leave commas inside the lyric as part of a single lyric line without bothering with double-quotes, you can adjust the dataframe parameter for field delimiters and use something else like a semicolon. E.g., pandas.dataframe_name.read_csv(sep=';'). I haven't implemented this personally so the approach may need some adjusting. Just be aware it can get dicey if you're trying to combine dataframes that are using different delimiters.
3. Starting with a file with just one column containing a song's lyrics, build a dataframe with more columns that include details about the song, to match our comprehensive lyrics dataframe. See cells below for the functions to do this.
4. Concatenate/append this one song's dataframe to our comprehensive lyrics dataframe.
5. Check our files and variables and then write the new and improved, extended comprehensive dataframe to our main lyrics data csv file. (Again, be careful when writing over big files. You can write to a temporary file to check accuracy beforehand; I used one called "testingoutput.csv".)

*If you want to inspect the lyrics I have built with this method, you can find them in the file tree. To find these through Jupyter, click on the Jupyter logo in the upper left corner. You can, of course, also view these in the file directory in the github repository.*

In [2]:
# Load datasets

# Comprehensive, main lyrics list to build upon
lyrics = pd.read_csv("twenty_one_pilots_lyrics_2009-2024.csv")

# Song to add to the comprehensive lyrics list
one_song = pd.read_csv("one_song_lyrics_only.csv")

# Don't forget to include a "lyric" at the top (first line) of your "one_song_lyrics_only.csv"!
# CSV files are read with the first line as the column name/s.

In [3]:
# Inspect the first few rows of our target comprehensive dataframe
lyrics.head()

,album_name,track_title,track_n,lyric,line
0,Johnny Boy - EP,Johnny Boy,1,He stays home from work this time,1
1,Johnny Boy - EP,Johnny Boy,1,He never really told his wife,2
2,Johnny Boy - EP,Johnny Boy,1,He never really told a lie but this time he de...,3
3,Johnny Boy - EP,Johnny Boy,1,It's alright,4
4,Johnny Boy - EP,Johnny Boy,1,No one really knows his mind and no one knows ...,5


In [4]:
# A sample of what we can do with our data
# Look at the names of all songs in this dataframe, without repeats
print(lyrics.track_title.unique())

['Johnny Boy' 'Air Catcher' 'Time to Say Goodbye' 'Addict with a Pen'
 'Friend, Please' 'Taxi Cab' 'Implicit Demand for Proof' 'Fall Away'
 'The Pantaloon' 'March to the Sea' 'Oh, Ms. Believer' 'Trapdoor'
 'A Car, a Torch, a Death' 'Before You Start Your Day'
 'Isle of Flightless Birds' 'Guns for Hands' 'Holding on to You'
 'Ode to Sleep' 'Slowtown' 'Car Radio' 'Forest' 'Glowing Eyes'
 'Kitchen Sink' 'Anathema' 'Lovely' 'Ruby' 'Trees' 'Be Concerned' 'Clear'
 'House of Gold' 'Two']


In [5]:
# Inspect the first few rows of our lyrics to build upon
one_song.head()

,lyric
0,I wake up fine and dandy
1,but then by the time I find it handy
2,To rip my heart apart and start planning my cr...
3,"I go up, up, up, up, up to the ceiling"
4,Then I feel my soul start leaving


In [6]:
# Get info about the song's dataframe, double check that there are the same # of entries as there are objects, and no null objects
one_song.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lyric   86 non-null     object
dtypes: object(1)
memory usage: 816.0+ bytes


In [7]:
# Get info about our target dataframe, noticing again the columns we want to add to our one song.
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1841 entries, 0 to 1840
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   album_name   1841 non-null   object
 1   track_title  1841 non-null   object
 2   track_n      1841 non-null   int64 
 3   lyric        1841 non-null   object
 4   line         1841 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 72.0+ KB


In [8]:
# Build new information around our song lyrics to match the target dataframe.

# First, we can use variables to make it even easier and more obvious where to modify content per song:
one_song_album_name = "Vessel"
one_song_track_title = "Ode to Sleep"
one_song_track_n = 1

# Then, we'll assign these values to cells in our dataframe.

# If you don't care about matching columns to the same indices, you can simply assign, e.g.:
# one_song['album_name'] = one_song_album
# lyrics['track_n'] = one_song_track_num
# If you do care about that, you can use .insert(), like this:
one_song.insert(0,'album_name', one_song_album_name)
one_song.insert(1,'track_title', one_song_track_title)
one_song.insert(2,'track_n', one_song_track_n)
# You can add what is basically an incrementing index per lyric line with a range func, like this:
one_song.insert(4, 'line', range(1, 1+len(one_song)))

# NOTE that you can run a simple assignment many times without any consequence, 
# because it will just re-assign the same values,
# but .insert() creates a new column with the content and can only be run for a unique identifier once per session.
# So don't run a cell with .insert() more than once.

In [9]:
# Now take another look at our updated one song's dataframe
one_song.head()

,album_name,track_title,track_n,lyric,line
0,Vessel,Ode to Sleep,1,I wake up fine and dandy,1
1,Vessel,Ode to Sleep,1,but then by the time I find it handy,2
2,Vessel,Ode to Sleep,1,To rip my heart apart and start planning my cr...,3
3,Vessel,Ode to Sleep,1,"I go up, up, up, up, up to the ceiling",4
4,Vessel,Ode to Sleep,1,Then I feel my soul start leaving,5


In [10]:
# Write the new song's dataframe to its own csv, if wanted
one_song.to_csv('one_song_lyrics_finished.csv', index=False)

In [11]:
# Concatenate new song's dataframe with the comprehensive list
combined_lyrics = pd.concat([lyrics,one_song])
combined_lyrics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1927 entries, 0 to 85
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   album_name   1927 non-null   object
 1   track_title  1927 non-null   object
 2   track_n      1927 non-null   int64 
 3   lyric        1927 non-null   object
 4   line         1927 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 90.3+ KB


You can also concatenate it to a csv for just that album, if you'd like to have that on hand too.
Use a similar process to add these kinds of extra pieces to your dataset.

In [12]:
#combined_lyrics.to_csv('testingoutput.csv', index=False)

## STOP here!
Check your one_song_lyrics_finished.csv or testingoutput.csv file before making your csv edits permanent! If there is an error, you will need to fix and then read the file and manipulate the data again.

In [13]:
# If this test output looks good, make the change permanent by overwriting your main file.
combined_lyrics.to_csv('twenty_one_pilots_lyrics_2009-2024.csv', index=False)

### Now you can check your csv files to see the new data! 
If it all looks good, proceed to clean up your tracks and prepare to repeat the process for your next song. (Technically you don't have to do this since you'll be overwriting files when you start over anyway; it's more good practice and saves on space if you're not coming back to this for a while.) You can either do this manually by just deleting the file or deleting the contents of the file (which can be easy and fast enough to do by hand, since you're going to be copying and pasting into these files again anyway), OR you can wipe files through jupyter, by truncating with something like this:

In [14]:
# Delete the triple quotes to automate this feature.
"""f = open("testingoutput.csv", "w+")
f.close()
g = open("one_song_lyrics_only.csv", "w+")
g.close()
h = open("one_song_lyrics_finished.csv", "w+")
h.close()"""

'f = open("testingoutput.csv", "w+")\nf.close()\ng = open("one_song_lyrics_only.csv", "w+")\ng.close()\nh = open("one_song_lyrics_finished.csv", "w+")\nh.close()'

### 2. Start Analyzing!